# Winning Jeopardy

Jeopardy is a popular TV show in the US where participants answer questions to win money. It's been running for a few decades, and is a major force in popular culture. 

Let's say we want to compete on Jeopardy, and we're looking for any edge we can get to win. In this project, we'll work with a dataset of Jeopardy questions to figure out some patterns in the questions that could help us win.

**Project Goal**
In this project, we'll work with a dataset of Jeopardy questions to figure out some patterns in the questions that could help us win.

This project is part of the guided project series on [dataquest](http://dataquest.io/).

## Jeopardy Questions

Let's explore the jeopardy questions. We will see that the dataset contains the following columns:

* `Show Number` -- the Jeopardy episode number of the show this question was in.
* `Air Date` -- the date the episode aired.
* `Round` -- the round of Jeopardy that the question was asked in. Jeopardy has several rounds as each episode progresses.
* `Category` -- the category of the question.
* `Value` -- the number of dollars answering the question correctly is worth.
* `Question` -- the text of the question.
* `Answer` -- the text of the answer.

In [1]:
import pandas as pd
from string import punctuation
import re

# read in the dataset
jeopardy = pd.read_csv('jeopardy.csv')
jeopardy.columns

FileNotFoundError: [Errno 2] File b'jeopardy.csv' does not exist: b'jeopardy.csv'

In [ ]:
# clean column names from additional spaces
jeopardy.columns = jeopardy.columns.str.replace(' ', '')
jeopardy.columns

In [ ]:
jeopardy.shape

In [ ]:
jeopardy.head()

Our dataset contains 216930 questions.

## Normalizing Text

Before we start with our analysis, we need to normalize the text, that is ensure lowercase words and remove punctuation. There are two ways we can go about this. Let's see which one is more efficient.

In [ ]:
# function to normalize the text using python string library
def normalize_text(s):
    s = str(s).lower()
    s = s.translate(str.maketrans('', '', punctuation))
    return s

# function to normalize the text using regex
def normalize_text_reg(s):
    s = str(s).lower()
    s = re.sub("[^A-Za-z0-9\s]", "", s)
    return s

s = 'Hello.?! World.*'
%timeit normalize_text(s)
%timeit normalize_text_reg(s)

We can observe that using the function with regular expression seems to be much faster. We will use it to treat our `Question` and `Answer` columns.

In [ ]:
# normalize our columns
jeopardy['clean_question'] = jeopardy['Question'].apply(normalize_text_reg)
jeopardy['clean_answer'] = jeopardy['Answer'].apply(normalize_text_reg)

## Normalizing Columns

Now that we have normalized the string columns, let's also normalize `Value` and `Air Date` columns. We will remove the $ sign from the `Value` column and convert the column to numeric; we will also convert `Air Date` to datetime.

In [ ]:
# function to normalize value column 
def normalize_value(s):
    s = re.sub("[^A-Za-z0-9\s]", "", s)
    try:
        s = int(s)
    except:
        s = 0
    return s

# clean value column
jeopardy['clean_value'] = jeopardy['Value'].apply(normalize_value)
jeopardy['AirDate'] = pd.to_datetime(jeopardy['AirDate'])

## Answers in Questions

In order to figure out whether to study past questions, study general knowledge, or not study it all, it would be helpful to figure out two things:

* How often the answer is deducible from the question.
* How often new questions are repeats of older questions.

We can answer the first question by seeing how many times words in the answer also occur in the question. We can answer the 2nd question by seeing how often complex words (> 6 characters) reoccur. Let's tackle the first question first.

In [ ]:
# function to count words from answer in the question
def count_terms(row):
    # split the column values by space
    split_answer = row['clean_answer'].split()
    split_question = row['clean_question'].split()    
    
    # initialize count of words
    match_count = 0
    
    # remove 'the' from answers
    if 'the' in split_answer:
        split_answer.remove('the')
    
    # check if answer is 0
    if len(split_answer) == 0:
        return 0
    
    # count ocurrences of answer values in question
    for i in split_answer:
        if i in split_question:
            match_count += 1
    
    # return the calculated count
    return match_count / len(split_answer)
    
# count the words
jeopardy['answer_in_question'] = jeopardy.apply(count_terms, axis=1)

# calculate the mean
jeopardy['answer_in_question'].mean()

We can see that on average the answer shows up in the question around 6% of the time. This tells us that its quite difficult to deduce the answer from the question.

Let's move to tackling our second question, where we need to find out how often new questions are repeats of older ones.

In [ ]:
question_overlap = []
terms_used = set()

# sorty jeopary values in ascending order
jeopardy.sort_values('AirDate', inplace=True)

# iterate through rows
for i, row in jeopardy.iterrows():
    # split the question by space
    split_question = row['clean_question'].split()
    
    # remove words with less than 6 characters (keep the ones more than)
    split_question = [word for word in split_question if len(word) >= 6]
    
    # initialize count
    match_count = 0
    
    # loop through words in split_question: count matches
    for w in split_question:
        if w in terms_used:
            match_count += 1

        # add words to terms used set
        terms_used.add(w)
            
    if len(split_question) > 0:
        match_count /= len(split_question)
        
    question_overlap.append(match_count)
    
jeopardy['question_overlap'] = question_overlap
jeopardy['question_overlap'].mean()

Our results show that there is about 87% overlap between terms in new questions vs terms in old questions. This doesn't actually suggest us to actually study past questions, since it only looks at terms instead of full questions or even phrases. Nevertheless, it tells us to explore the subject of reocurrence even more.